In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder, StandardScaler
import lightgbm as lgbm

In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
pd.set_option('display.max_columns', len(train.columns))

In [3]:
label = LabelEncoder()
train['gender'] = label.fit_transform(train['gender'])
train['age_group'] = label.fit_transform(train['age_group'])
train['race'] = label.fit_transform(train['race'])
train['religion'] = label.fit_transform(train['religion'])

test['gender'] = label.fit_transform(test['gender'])
test['age_group'] = label.fit_transform(test['age_group'])
test['race'] = label.fit_transform(test['race'])
test['religion'] = label.fit_transform(test['religion'])

In [4]:
train

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43375,1.0,324,5.0,1067,5.0,359,1.0,709,1.0,742,1.0,330,5.0,541,5.0,1376,2.0,1397,5.0,1250,2.0,762,5.0,517,4.0,3614,1.0,521,5.0,705,4.0,633,1.0,330,1.0,512,5.0,347,5.0,588,2,3,1,2,1,1,1,6,3,2,2,1,5,2,4,5,4,1,2,2,1,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,1
4486,1.0,534,5.0,1555,5.0,2024,1.0,1097,1.0,3077,3.0,6721,1.0,1306,4.0,2968,5.0,1998,5.0,2756,2.0,3429,5.0,2395,5.0,1745,1.0,584,5.0,1857,5.0,2894,2.0,3761,1.0,2717,3.0,2802,5.0,1299,1,2,2,4,1,1,1,1,7,1,0,2,2,0,6,1,5,5,6,3,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1
10262,5.0,609,1.0,749,2.0,624,1.0,1833,3.0,1474,5.0,728,4.0,1107,3.0,1743,5.0,3008,3.0,1649,5.0,870,1.0,1551,1.0,989,5.0,347,1.0,824,1.0,1445,2.0,884,5.0,744,1.0,899,4.0,963,2,2,1,3,0,1,1,6,5,0,1,0,5,3,6,0,2,1,6,2,1,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0
14088,4.0,182,1.0,2969,1.0,1955,4.0,4630,1.0,1321,3.0,2345,4.0,850,1.0,3559,5.0,6761,1.0,8315,5.0,614,1.0,1751,1.0,1919,4.0,96,4.0,1251,5.0,261,4.0,548,4.0,2576,2.0,877,5.0,967,1,3,2,3,1,1,1,5,7,2,2,4,5,0,1,0,2,0,4,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
13398,4.0,549,5.0,1679,5.0,481,3.0,595,1.0,1202,1.0,348,1.0,422,2.0,2453,1.0,1051,5.0,4733,3.0,2705,5.0,773,4.0,775,2.0,1066,5.0,824,1.0,750,4.0,535,4.0,1000,4.0,964,5.0,683,2,2,1,4,0,1,1,6,10,6,3,6,2,0,6,2,0,6,4,2,1,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16511,1.0,312,5.0,965,5.0,499,1.0,1188,1.0,437,1.0,319,3.0,14430,3.0,1435,1.0,2406,5.0,1222,3.0,650,5.0,8213,4.0,1231,3.0,454,5.0,764,5.0,480,1.0,526,1.0,598,5.0,461,5.0,729,2,2,1,6,0,1,1,2,10,4,3,3,5,0,1,2,2,0,3,2,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1
33507,3.0,474,4.0,1033,5.0,602,2.0,703,1.0,782,3.0,410,1.0,388,3.0,865,4.0,1533,5.0,639,4.0,1302,2.0,725,5.0,1104,2.0,638,4.0,480,5.0,984,3.0,316,2.0,688,5.0,528,5.0,732,1,1,2,4,1,3,1,6,8,5,1,1,6,2,5,4,6,3,6,1,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1
44917,2.0,617,4.0,1262,2.0,709,1.0,1063,2.0,1728,1.0,404,4.0,635,2.0,6379,2.0,2630,4.0,1362,5.0,1714,5.0,642,4.0,538,1.0,1432,5.0,3183,2.0,1183,4.0,435,1.0,729,5.0,751,3.0,976,3,4,1,6,1,1,2,6,0,1,2,0,5,1,2,1,2,1,5,2,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1


In [6]:
drop_list = ['voted']

train_x = train.drop(drop_list, axis=1)
train_y = train['voted']

In [7]:
train_y

index
43375    1
4486     1
10262    1
14088    0
13398    1
        ..
16511    1
33507    0
44917    1
36126    1
42340    1
Name: voted, Length: 36425, dtype: int64

In [9]:
model = lgbm.LGBMClassifier(n_estimators=500)
model.fit(train_x, train_y)

LGBMClassifier(n_estimators=500)

In [14]:
pred = model.predict(test)

In [18]:
train_x

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43375,1.0,324,5.0,1067,5.0,359,1.0,709,1.0,742,1.0,330,5.0,541,5.0,1376,2.0,1397,5.0,1250,2.0,762,5.0,517,4.0,3614,1.0,521,5.0,705,4.0,633,1.0,330,1.0,512,5.0,347,5.0,588,2,3,1,2,1,1,1,6,3,2,2,1,5,2,4,5,4,1,2,2,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,1
4486,1.0,534,5.0,1555,5.0,2024,1.0,1097,1.0,3077,3.0,6721,1.0,1306,4.0,2968,5.0,1998,5.0,2756,2.0,3429,5.0,2395,5.0,1745,1.0,584,5.0,1857,5.0,2894,2.0,3761,1.0,2717,3.0,2802,5.0,1299,1,2,2,4,1,1,1,1,7,1,0,2,2,0,6,1,5,5,6,3,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1
10262,5.0,609,1.0,749,2.0,624,1.0,1833,3.0,1474,5.0,728,4.0,1107,3.0,1743,5.0,3008,3.0,1649,5.0,870,1.0,1551,1.0,989,5.0,347,1.0,824,1.0,1445,2.0,884,5.0,744,1.0,899,4.0,963,2,2,1,3,0,1,1,6,5,0,1,0,5,3,6,0,2,1,6,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0
14088,4.0,182,1.0,2969,1.0,1955,4.0,4630,1.0,1321,3.0,2345,4.0,850,1.0,3559,5.0,6761,1.0,8315,5.0,614,1.0,1751,1.0,1919,4.0,96,4.0,1251,5.0,261,4.0,548,4.0,2576,2.0,877,5.0,967,1,3,2,3,1,1,1,5,7,2,2,4,5,0,1,0,2,0,4,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
13398,4.0,549,5.0,1679,5.0,481,3.0,595,1.0,1202,1.0,348,1.0,422,2.0,2453,1.0,1051,5.0,4733,3.0,2705,5.0,773,4.0,775,2.0,1066,5.0,824,1.0,750,4.0,535,4.0,1000,4.0,964,5.0,683,2,2,1,4,0,1,1,6,10,6,3,6,2,0,6,2,0,6,4,2,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16511,1.0,312,5.0,965,5.0,499,1.0,1188,1.0,437,1.0,319,3.0,14430,3.0,1435,1.0,2406,5.0,1222,3.0,650,5.0,8213,4.0,1231,3.0,454,5.0,764,5.0,480,1.0,526,1.0,598,5.0,461,5.0,729,2,2,1,6,0,1,1,2,10,4,3,3,5,0,1,2,2,0,3,2,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1
33507,3.0,474,4.0,1033,5.0,602,2.0,703,1.0,782,3.0,410,1.0,388,3.0,865,4.0,1533,5.0,639,4.0,1302,2.0,725,5.0,1104,2.0,638,4.0,480,5.0,984,3.0,316,2.0,688,5.0,528,5.0,732,1,1,2,4,1,3,1,6,8,5,1,1,6,2,5,4,6,3,6,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1
44917,2.0,617,4.0,1262,2.0,709,1.0,1063,2.0,1728,1.0,404,4.0,635,2.0,6379,2.0,2630,4.0,1362,5.0,1714,5.0,642,4.0,538,1.0,1432,5.0,3183,2.0,1183,4.0,435,1.0,729,5.0,751,3.0,976,3,4,1,6,1,1,2,6,0,1,2,0,5,1,2,1,2,1,5,2,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1
